# RAG (Retrieval-Augmented Generation) System using Llama 3 405B Model
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1o0X6SXgwjkZ12aGPra7_KYMfPKfZhHCD?usp=sharing#scrollTo=ruXVpm7Op6bL)

####Enhancing language model outputs with relevant external knowledge
####Combining the power of large language models with dynamic information retrieval

## Install required dependencies

### Note: In a Jupyter notebook, you would use !pip install or %pip install


In [ ]:
!pip install langchain-openai faiss-cpu langchain openai requests numpy tiktoken langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 725.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

##Import necessary libraries


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from google.colab import userdata
from langchain_openai import ChatOpenAI
from google.colab import userdata
import requests
import numpy as np
import faiss
import os




## Initialize the language model (LLM)
### We're using the Llama 3 405B model hosted on Fireworks.ai

In [ ]:
llm = ChatOpenAI(
    model="accounts/fireworks/models/llama-v3p1-405b-instruct",
    openai_api_key=userdata.get("FIREWORKS_API_KEY"),
    openai_api_base="https://api.fireworks.ai/inference/v1"
)

## Create embeddings for the documents
### Note: We're using OpenAI's embeddings here. You might want to use a different embedding model.


In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=userdata.get("OPENAI_API_KEY"))


## Prepare your document collection using WebBaseLoader
### We're using WebBaseLoader to load content from specified URLs


In [ ]:
urls = [
    "https://www.theverge.com/2024/7/23/24204055/meta-ai-llama-3-1-open-source-assistant-openai-chatgpt",
    # Add more URLs as needed
]
loader = WebBaseLoader(urls)
documents = loader.load()

## Split the documents into chunks
### This makes it easier to process and retrieve relevant information

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Create embeddings for the documents and store them in a FAISS vector database


In [ ]:
db = FAISS.from_documents(texts, embeddings)

## Combine retrieval and generation
#### This sets up the RAG system, combining the FAISS database for retrieval
#### with the Llama 3 405B model for generation

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
)

## Use the RAG system with example queries
#### You can modify these queries or add more as needed

### Example Query 1

In [ ]:
query = "Meta's new Llama 3.1 model outperforms which models?"
result = qa_chain.invoke({"query": query})
print(result['result'])

According to the text, Meta's Llama 3.1 model outperforms OpenAI's GPT-4o and Anthropic's Claude 3.5 Sonnet on certain benchmarks.


### Example Query 2

In [ ]:
query = "give me top features of Llama 3.1 model"
result = qa_chain.invoke({"query": query})
print(result['result'])

Based on the text, here are some of the top features of the Llama 3.1 model:

1. Complex problem-solving: Llama 3.1 is capable of integrating with a search engine API to "retrieve information from the internet based on a complex query and call multiple tools in succession in order to complete your tasks".

2. Advanced natural language processing: It can understand and respond to complex queries and is capable of generating human-like text.

3. Image generation: Meta AI, powered by Llama 3.1, includes a new feature called "Imagine Me" that can scan a face through a phone's camera and then let users insert their likeness into images it generates.

4. Multilingual support: Llama 3.1 will be updated to support new languages, including French, German, Hindi, Italian, and Spanish.

5. Availability across multiple platforms: Llama 3.1 will be first accessible through WhatsApp and the Meta AI website in the US, followed by Instagram and Facebook in the coming weeks, and also on the Quest heads